In [210]:
import pandas as pd
import requests
import json

def statement(ticker,statement,period):
    if statement =='IS':
        temp='income-statement'
    elif statement =='BS':
        temp='balance-sheet-statement'
    else:
        temp='cash-flow-statement'
    if period=='Q':
        temp2 = 'quarter'
    else:
        temp2 = 'annual'
    bs = requests.get(f'https://financialmodelingprep.com/api/v3/financials/'+temp+'/'+ticker+'?period='+temp2)
    bs = bs.json()

    bs = bs['financials']
    bs = pd.DataFrame.from_dict(bs)
    bs = bs.T
    bs.columns = bs.iloc[0]

    bs = bs.iloc[1:]
    bs = bs.iloc[:,:4]
    
    return bs

In [211]:
def net_income(ticker):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    return float(df.loc['Net Income'][0])

def roa(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['Total assets'][0]),float(df.loc['Total assets'][1])
    av_assets=(p+q)/2
    return net_income(ticker)/av_assets

def ocf(ticker):
    df=pd.DataFrame(statement(ticker,'CF','A'))
    return float(df.loc['Operating Cash Flow'][0])

def ltdebt(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['Long-term debt'][0]),float(df.loc['Long-term debt'][1])
    return q-p

def current_ratio(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['Total current assets'][0]),float(df.loc['Total current assets'][1])
    r,s=float(df.loc['Total current liabilities'][0]),float(df.loc['Total current liabilities'][1])
    current_ratio1,current_ratio2=p/r,q/s
    return current_ratio1-current_ratio2

def new_shares(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['Retained earnings (deficit)'][0]),float(df.loc['Retained earnings (deficit)'][1])
    r,s=float(df.loc['Total shareholders equity'][0]),float(df.loc['Total shareholders equity'][1])
    return (s-q)-(r-p)

def gross_margin(ticker):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    p,q=float(df.loc['Gross Margin'][0]),float(df.loc['Gross Margin'][1])
    return p-q

def Asset_Turnover_Ratio(ticker):
    df1=pd.DataFrame(statement(ticker,'IS','A'))
    df2=pd.DataFrame(statement(ticker,'BS','A'))
    p,q,s=float(df2.loc['Total assets'][0]),float(df2.loc['Total assets'][1]),float(df2.loc['Total assets'][2])
    av_assets1=(p+q)/2
    av_assets2=(q+s)/2
    atr1=float(df1.loc['Revenue'][0])/av_assets1
    atr2=float(df1.loc['Revenue'][1])/av_assets2
    return atr1-atr2

def Piotroski_Score(ticker):
    Pscore=0
    if net_income(ticker)>0:
        Pscore+=1
    if roa(ticker)>0:
        Pscore+=1
    if ocf(ticker)>0:
        Pscore+=1
    if ocf(ticker)>net_income(ticker):
        Pscore+=1
    if ltdebt(ticker)>0:
        Pscore+=1
    if current_ratio(ticker)>0:
        Pscore+=1
    if new_shares(ticker)>0:
        Pscore+=1
    if gross_margin(ticker)>0:
        Pscore+=1
    if Asset_Turnover_Ratio(ticker)>0:
        Pscore+=1
    return Pscore

In [225]:
tickers=['AAPL','GOOG','TSLA','F','MSFT','MS']
data2={}
for ticker in tickers:
    data2[ticker]=Piotroski_Score(ticker)
    data=pd.DataFrame(data2,index=[0])    
data

,AAPL,GOOG,TSLA,F,MSFT,MS
0,7,4,4,5,7,6
